Importing necessary libraries


In [ ]:
import json
import spacy
import pandas as pd
import ast 
import json
import spacy
import random
import logging
import srsly
from spacy.gold import docs_to_json, biluo_tags_from_offsets, spans_from_biluo_tags
import os
from __future__ import unicode_literals, print_function
import plac
import random
from pathlib import Path
from tqdm import tqdm
from spacy.scorer import Scorer
from spacy.gold import GoldParse


In [ ]:
from google.colab import drive
drive.mount('/content/drive') #mount drive 

Mounted at /content/drive


import train and test data


In [ ]:
train= ['/content/1-3.txt', '/content/10-12.txt','/content/7-9.txt','/content/4-6.txt','/content/49-50.txt',"/content/46-48.txt","/content/43-45.txt","/content/37-39.txt","/content/34-36.txt","/content/28-30.txt","/content/25-27.txt","/content/22-24.txt","/content/16-18.txt","/content/13-15.txt"]

In [ ]:
#reading annotated files 
Train_data=[] 
for fname in train:
  f=open(fname,"r")
  d=(f.read())
  y=ast.literal_eval(d)
  Train_data.extend(y)



In [ ]:
test = ['/content/40-42.txt','/content/31-33.txt','/content/19-21.txt']

In [ ]:
Test_data=[] 
for fname in test:
  f=open(fname,"r")
  d=(f.read())
  y=ast.literal_eval(d)
  Test_data.extend(y)
print(type(Test_data))

<class 'list'>


Data processing


In [36]:
## Training Data

In [ ]:
#load pretrained spacy english model 
nlp = spacy.load('en')
#covert annotated train data to json file
docs = []
for text, annot in Train_data:
    doc = nlp(text)
    tags = biluo_tags_from_offsets(doc, annot['entities'])
    entities = spans_from_biluo_tags(doc, tags)
    doc.ents = entities
    docs.append(doc)

srsly.write_json("spacy_format1.json", [docs_to_json(docs)])
               

In [ ]:
#covert annotated test data to json file 
docs = []
for text, annot in Test_data:
    doc = nlp(text)
    tags = biluo_tags_from_offsets(doc, annot['entities'])
    entities = spans_from_biluo_tags(doc, tags)
    doc.ents = entities
    docs.append(doc)

srsly.write_json("spacy_format2.json", [docs_to_json(docs)])

In [ ]:
!python -m spacy train en save-model './spacy_format1.json' './spacy_format2.json' #train model and save to directory(replace "save-model" with your directory name)


✔ Created output directory: save-model
Training pipeline: ['tagger', 'parser', 'ner']
Starting with blank model 'en'
Counting training words (limit=0)
/usr/lib/python3.7/runpy.py:193: UserWarning: [W022] Training a new part-of-speech tagger using a model with no lemmatization rules or data. This means that the trained model may not be able to lemmatize correctly. If this is intentional or the language you're using doesn't have lemmatization data, you can ignore this warning by setting SPACY_WARNING_IGNORE=W022. If this is surprising, make sure you have the spacy-lookups-data package installed.
  "__main__", mod_spec)

Itn  Tag Loss    Tag %    Dep Loss    UAS     LAS    NER Loss   NER P   NER R   NER F   Token %  CPU WPS
---  ---------  --------  ---------  ------  ------  ---------  ------  ------  ------  -------  -------
  1   4781.032    68.359  10021.490  38.466   9.330   2219.393  35.484   6.395  10.837  100.000     6296
  2   2124.131    83.111   8129.127  55.700  19.740   1168.

Testing Model 


In [ ]:
sentence1=" The Nigerian police say four officers were killed and one is still missing after they clashed with an armed criminal gang in the Birnin-Gwari area of Kaduna state.Police Spokesperson\\
Frank Mba in a statement on Sunday said the officers were ambushed by about 100 banditsas they \
returned from an operation in neighbouring Niger state.The police say tens of gunmen were also killed in the clashes on Friday.The police statement denies initial reports that 18 officers were abducted during the attack.\
It says only 16 officers were involved in the incident and that the bodies of four of them and their weapons had\
 been recovered, 11 survived while efforts are being intensified to rescue the officer still missing.It’s not clear whether the missing officer\
 had been abducted by the armed men.Meanwhile, in other separate attacks on Sunday also in Kaduna state, the authorities confirmed at least five people including an 80-year-old woman and a local traditional ruler were ki\
In a previous attack within the week at least 18 people including mothers and their babies were kidnapped in a village in Birnin-Gwari area of the state. They’re yet to be released "

In [ ]:
model= spacy.load("/content/save-model/model-best")#loading the best model
result1 = model(sentence1)
for ent in result1.ents: 
    print(ent.text, ent.start_char, ent.end_char, ent.label_)
spacy.displacy.render(result1, style='ent',jupyter=True)

Nigerian 5 13 NORP
police 14 20 PERSON
four 25 29 CARDINAL
one 55 58 CARDINAL
Kaduna 151 157 NORP
Mba 190 193 NORP
Sunday 212 218 DATE
about 100 254 263 CARDINAL
Niger 322 327 GPE
police 338 344 PERSON
tens 349 353 CARDINAL
Friday 399 405 DATE
police 410 416 PERSON
18 455 457 CARDINAL
only 16 507 514 CARDINAL
four 577 581 CARDINAL
11 628 630 CARDINAL
Sunday 830 836 DATE
Kaduna 845 851 NORP
at least five 885 898 CARDINAL
80-year-old 919 930 DATE
a previous 977 987 DATE
the week 1002 1010 DATE
at least 18 people 1011 1029 EVENT
a village 1083 1092 DATE
Birnin 1096 1102 GPE


In [ ]:
#evaluating the f-score 
def Eval(ner_model,test_data):
  scorer = Scorer()
  try:
    for input_, annot in test_data:
      doc_gold_text = ner_model.make_doc(input_)
      gold = GoldParse(doc_gold_text, entities=annot['entities'])
      pred_value = ner_model(input_)
      scorer.score(pred_value, gold)
  except Exception as e: print(e)
 
  pprint(scorer.scores)
 
Eval(model, Test_data)

{'ents_f': 57.40181268882175,
 'ents_p': 59.006211180124225,
 'ents_per_type': {'CARDINAL': {'f': 71.69811320754717,
                                'p': 82.6086956521739,
                                'r': 63.33333333333333},
                   'DATE': {'f': 53.333333333333336,
                            'p': 47.05882352941176,
                            'r': 61.53846153846154},
                   'EVENT': {'f': 55.55555555555556,
                             'p': 83.33333333333334,
                             'r': 41.66666666666667},
                   'FAC': {'f': 0.0, 'p': 0.0, 'r': 0.0},
                   'GPE': {'f': 78.26086956521739,
                           'p': 71.05263157894737,
                           'r': 87.09677419354838},
                   'LOC': {'f': 0.0, 'p': 0.0, 'r': 0.0},
                   'NORP': {'f': 64.0, 'p': 50.0, 'r': 88.88888888888889},
                   'ORDINAL': {'f': 50.0, 'p': 100.0, 'r': 33.33333333333333},
                   'ORG': {'f

In [34]:
sentence2= "A union representing lecturers in Nigeria has announced the suspension of a nine month strike that has crippled learning at public universities.Prof Biodun Ogunyemi, the national president of the Academic Staff Union of Universities, told the BBC that the union decided to suspend the strike\
after concrete agreements were reached with the federal government on Tuesday.\
Prof Ogunyemi said lecturers will return to the universities from Thursday but warned that the union would resume the strike action if the government failed to fulfil a return to work deal.\
The union has been pushing the government to implement a 2009 agreement that promised equipment for universities and an increase in lecturers pay.\
The union said students will be invited back to universities after measures are put in place to prevent the spread of Covid-19."


In [35]:
#testing the model on sentence 2
result2 = model(sentence2)
for ent in result2.ents: 
    print(ent.text, ent.start_char, ent.end_char, ent.label_)
spacy.displacy.render(result1, style='ent',jupyter=True)

union 2 7 ORG
lecturers 21 30 PERSON
Nigeria 34 41 GPE
nine month 76 86 DATE
strike 87 93 EVENT
universities 131 143 ORG
Biodun Ogunyemi 149 164 PERSON
president 179 188 PERSON
the Academic Staff Union of Universities 192 232 ORG
BBC 243 246 ORG
union 256 261 ORG
Tuesday 361 368 DATE
Prof Ogunyemi 369 382 PERSON
lecturers 388 397 PERSON
universities 417 429 ORG
Thursday 435 443 DATE
union 464 469 ORG
strike 487 493 EVENT
government 508 518 PERSON
union 562 567 ORG
government 589 599 PERSON
2009 615 619 DATE
universities 658 670 ORG
lecturers pay 690 703 PERSON
union 708 713 ORG
universities 752 764 ORG
Covid-19 822 830 EVENT
